In [111]:
import spacy
from spacy.tokens import Doc
from spacy.attrs import HEAD, DEP, POS
import pandas as pd
import numpy as np
import lmdb
from lmdb_embeddings.reader import LmdbEmbeddingsReader
import lmdb_embeddings.exceptions as exceptions
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Embedding, concatenate, Flatten, Dropout, SpatialDropout1D, TimeDistributed, Reshape, Lambda, LSTM, Input
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import to_categorical, np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
nlp=spacy.load('en_core_web_sm')
embeddings=LmdbEmbeddingsReader('data/lmdb_databases')
encoder=LabelEncoder()

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


<h3>Loading the data</h3>

In [39]:
semeval_emotions=pd.read_csv('data/semeval_emotions.csv')
semeval_emotions.rename({'0': 0, '1': 1}, axis='columns', inplace=True)
isear=pd.read_csv('data/isear_1.csv', header=-1)
isear=isear.drop(2 , axis=1)
isear[0][isear[0]=='guit']='guilt'
isear_plus_semeval=isear.append(semeval_emotions)
keep_emotions=['anger',  'fear', 'joy', 'sadness']
for index, row in isear_plus_semeval.iterrows():
    if row[0] not in keep_emotions:
        isear_plus_semeval.drop(index=index, axis=0, inplace=True)
isear_plus_semeval=isear_plus_semeval.reset_index(drop=True).sample(frac=1)
X_init=isear_plus_semeval[1]
y_init=isear_plus_semeval[0]

In [40]:
X_init

2289    When someone made me a compliment after an exa...
812              After 3 weeks I saw my girlfriend again.
2519    My sister came back home and reproached me for...
5452    #MTPDaily, @DonnyDeutsch that look you've been...
1790    A Maori language oral test, yeaterday - Althou...
1188                Every time I see a spider or a snake.
4449    If I spend even 5 minutes with you and you alr...
1492    Late one night I was pursued by gipsies who ha...
4897    I turn 25 in two weeks. I am so happy. 24 was ...
4717    @Alyssa_Milano @IvankaTrump #Alyssa Y don't U ...
3413    When a relative, who had come to our place, cr...
4746    Retweeted GunnySmith93 (@Stephen21Smith):\n\nD...
1705    I felt it when a girl who was dear to me lied ...
2320     Once I saw some children hunting a cat to death.
4474    @iAmSJ I'm so mad for our clients I'm furious ...
611     My father had been drinking and I feared him w...
2493                           When I talk to my friends.
5574          

In [210]:
y_init

2289        joy
812         joy
2519      anger
5452       fear
1790       fear
1188       fear
4449      anger
1492       fear
4897        joy
4717      anger
3413      anger
4746        joy
1705      anger
2320      anger
4474      anger
611        fear
2493        joy
5574    sadness
533       anger
3724    sadness
1285      anger
1372        joy
4236      anger
3375       fear
4878        joy
127         joy
2888       fear
4128        joy
4037    sadness
4463      anger
         ...   
5410       fear
4880        joy
3730       fear
1130        joy
840         joy
2095    sadness
4297       fear
4613      anger
463         joy
3635      anger
1212        joy
4319    sadness
3231        joy
2858    sadness
2535      anger
360     sadness
1605        joy
3247       fear
1844      anger
100     sadness
4309       fear
5067        joy
3532       fear
5378       fear
4354      anger
2585      anger
1629        joy
4092    sadness
2947       fear
5663    sadness
Name: 0, Length: 5908, d

In [7]:
def lmdb_vec_word(word):
    try:
        vector = embeddings.get_word_vector(word)
    except exceptions.MissingWordError:
        # 'google' is not in the database.
        return np.zeros(300, dtype='float32')
    return vector

In [8]:
def transform_y(y):
    encoder.fit(y)
    y=encoder.transform(y)
    y_1=np_utils.to_categorical(y)
    #y_1=np.reshape(y_1, (-1, 4, 1))
    return y_1

In [314]:
main_input=Input(shape=(6,300), name='main_input')
lstm_out=LSTM(8)(main_input)
auxiliary_output = Dense(4, activation='softmax', name='aux_output')(lstm_out)
auxiliary_input = Input(shape=(6,2), name='aux_input')
aux_lstm=LSTM(8)(auxiliary_input)
auxiliary_input_2 = Input(shape=(6,300), name='aux_input_2')
aux_lstm_2=LSTM(8)(auxiliary_input_2)
x=concatenate([lstm_out, aux_lstm, aux_lstm_2])
# We stack a deep densely-connected network on top
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
# And finally we add the main logistic regression layer
main_output = Dense(4, activation='softmax', name='main_output')(x)

In [315]:
model = Model(inputs=[main_input, auxiliary_input, auxiliary_input_2], outputs=[main_output, auxiliary_output])

In [316]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [185]:
main_input_train, auxiliary_input_train, auxiliary_input_2_train, y_1=transform_X_y(X_init, y_init, 1000)

In [257]:
main_input_train.shape

(18313, 6, 300)

In [232]:
main_train, main_test, y_1_train, y_1_test=train_test_split(main_input_train, y_1, random_state=1)
aux_1_train, aux_1_test, y_1_train, y_1_test=train_test_split(auxiliary_input_train, y_1, random_state=1)
aux_2_train, aux_2_test, y_1_train, y_1_test=train_test_split(auxiliary_input_2_train, y_1, random_state=1)

In [240]:
model.fit([main_train, aux_1_train, aux_2_train], [y_1_train,y_1_train], epochs=5)

Epoch 1/5


10464/13734 [=====================>........] - ETA: 32:11 - loss: 2.8060 - main_output_loss: 1.3982 - aux_output_loss: 1.4077 - main_output_acc: 0.2812 - aux_output_acc: 0.25 - ETA: 5:22 - loss: 2.8296 - main_output_loss: 1.4320 - aux_output_loss: 1.3976 - main_output_acc: 0.2083 - aux_output_acc: 0.2135 - ETA: 2:55 - loss: 2.8063 - main_output_loss: 1.4139 - aux_output_loss: 1.3924 - main_output_acc: 0.2273 - aux_output_acc: 0.227 - ETA: 2:00 - loss: 2.7919 - main_output_loss: 1.4027 - aux_output_loss: 1.3892 - main_output_acc: 0.2539 - aux_output_acc: 0.240 - ETA: 1:32 - loss: 2.7835 - main_output_loss: 1.3971 - aux_output_loss: 1.3864 - main_output_acc: 0.2574 - aux_output_acc: 0.256 - ETA: 1:14 - loss: 2.7839 - main_output_loss: 1.3974 - aux_output_loss: 1.3865 - main_output_acc: 0.2524 - aux_output_acc: 0.259 - ETA: 1:00 - loss: 2.7770 - main_output_loss: 1.3924 - aux_output_loss: 1.3846 - main_output_acc: 0.2627 - aux_output_acc: 0.269 - ETA: 51s - loss: 2.7715 - main_output_loss

10848/13734 [======================>.......] - ETA: 5s - loss: 1.8787 - main_output_loss: 0.8810 - aux_output_loss: 0.9977 - main_output_acc: 0.7500 - aux_output_acc: 0.562 - ETA: 4s - loss: 1.9225 - main_output_loss: 0.8878 - aux_output_loss: 1.0346 - main_output_acc: 0.6771 - aux_output_acc: 0.552 - ETA: 4s - loss: 2.0050 - main_output_loss: 0.9259 - aux_output_loss: 1.0791 - main_output_acc: 0.6364 - aux_output_acc: 0.534 - ETA: 4s - loss: 1.9921 - main_output_loss: 0.9101 - aux_output_loss: 1.0819 - main_output_acc: 0.6465 - aux_output_acc: 0.535 - ETA: 4s - loss: 2.0212 - main_output_loss: 0.9250 - aux_output_loss: 1.0962 - main_output_acc: 0.6399 - aux_output_acc: 0.529 - ETA: 4s - loss: 2.0378 - main_output_loss: 0.9379 - aux_output_loss: 1.0999 - main_output_acc: 0.6296 - aux_output_acc: 0.527 - ETA: 4s - loss: 2.0492 - main_output_loss: 0.9508 - aux_output_loss: 1.0984 - main_output_acc: 0.6211 - aux_output_acc: 0.527 - ETA: 4s - loss: 2.0452 - main_output_loss: 0.9503 - aux_o

13734/13734 [==============================] - ETA: 0s - loss: 1.9870 - main_output_loss: 0.9176 - aux_output_loss: 1.0694 - main_output_acc: 0.6305 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9873 - main_output_loss: 0.9180 - aux_output_loss: 1.0693 - main_output_acc: 0.6300 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9867 - main_output_loss: 0.9172 - aux_output_loss: 1.0695 - main_output_acc: 0.6298 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9850 - main_output_loss: 0.9166 - aux_output_loss: 1.0684 - main_output_acc: 0.6296 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9858 - main_output_loss: 0.9172 - aux_output_loss: 1.0686 - main_output_acc: 0.6291 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9858 - main_output_loss: 0.9168 - aux_output_loss: 1.0690 - main_output_acc: 0.6290 - aux_output_acc: 0.544 - ETA: 0s - loss: 1.9844 - main_output_loss: 0.9156 - aux_output_loss: 1.0687 - main_output_acc: 0.6295 - aux_output_acc: 0.545 - ETA: 0s - loss: 1.9824 - main_output_loss: 0.9147 - aux_o

10496/13734 [=====================>........] - ETA: 5s - loss: 1.4203 - main_output_loss: 0.6400 - aux_output_loss: 0.7803 - main_output_acc: 0.8125 - aux_output_acc: 0.750 - ETA: 4s - loss: 1.6598 - main_output_loss: 0.7463 - aux_output_loss: 0.9135 - main_output_acc: 0.7344 - aux_output_acc: 0.635 - ETA: 4s - loss: 1.7228 - main_output_loss: 0.7755 - aux_output_loss: 0.9474 - main_output_acc: 0.7135 - aux_output_acc: 0.612 - ETA: 4s - loss: 1.8325 - main_output_loss: 0.8213 - aux_output_loss: 1.0112 - main_output_acc: 0.6823 - aux_output_acc: 0.560 - ETA: 4s - loss: 1.8481 - main_output_loss: 0.8252 - aux_output_loss: 1.0229 - main_output_acc: 0.6889 - aux_output_acc: 0.561 - ETA: 4s - loss: 1.8770 - main_output_loss: 0.8390 - aux_output_loss: 1.0380 - main_output_acc: 0.6719 - aux_output_acc: 0.550 - ETA: 4s - loss: 1.8587 - main_output_loss: 0.8315 - aux_output_loss: 1.0272 - main_output_acc: 0.6742 - aux_output_acc: 0.560 - ETA: 3s - loss: 1.8640 - main_output_loss: 0.8329 - aux_o

10752/13734 [======================>.......] - ETA: 5s - loss: 1.7279 - main_output_loss: 0.7066 - aux_output_loss: 1.0213 - main_output_acc: 0.6562 - aux_output_acc: 0.593 - ETA: 4s - loss: 1.7214 - main_output_loss: 0.7221 - aux_output_loss: 0.9993 - main_output_acc: 0.7188 - aux_output_acc: 0.619 - ETA: 4s - loss: 1.6997 - main_output_loss: 0.7101 - aux_output_loss: 0.9896 - main_output_acc: 0.7159 - aux_output_acc: 0.610 - ETA: 4s - loss: 1.7099 - main_output_loss: 0.7258 - aux_output_loss: 0.9841 - main_output_acc: 0.7090 - aux_output_acc: 0.599 - ETA: 4s - loss: 1.7255 - main_output_loss: 0.7306 - aux_output_loss: 0.9949 - main_output_acc: 0.7068 - aux_output_acc: 0.599 - ETA: 4s - loss: 1.7184 - main_output_loss: 0.7359 - aux_output_loss: 0.9824 - main_output_acc: 0.7067 - aux_output_acc: 0.607 - ETA: 4s - loss: 1.7314 - main_output_loss: 0.7402 - aux_output_loss: 0.9912 - main_output_acc: 0.6986 - aux_output_acc: 0.599 - ETA: 4s - loss: 1.7101 - main_output_loss: 0.7298 - aux_o

13734/13734 [==============================] - ETA: 0s - loss: 1.6953 - main_output_loss: 0.7241 - aux_output_loss: 0.9712 - main_output_acc: 0.7200 - aux_output_acc: 0.597 - ETA: 0s - loss: 1.6945 - main_output_loss: 0.7234 - aux_output_loss: 0.9711 - main_output_acc: 0.7206 - aux_output_acc: 0.597 - ETA: 0s - loss: 1.6940 - main_output_loss: 0.7230 - aux_output_loss: 0.9710 - main_output_acc: 0.7212 - aux_output_acc: 0.597 - ETA: 0s - loss: 1.6954 - main_output_loss: 0.7236 - aux_output_loss: 0.9719 - main_output_acc: 0.7210 - aux_output_acc: 0.597 - ETA: 0s - loss: 1.6980 - main_output_loss: 0.7250 - aux_output_loss: 0.9730 - main_output_acc: 0.7201 - aux_output_acc: 0.596 - ETA: 0s - loss: 1.6970 - main_output_loss: 0.7241 - aux_output_loss: 0.9728 - main_output_acc: 0.7209 - aux_output_acc: 0.596 - ETA: 0s - loss: 1.6989 - main_output_loss: 0.7258 - aux_output_loss: 0.9731 - main_output_acc: 0.7204 - aux_output_acc: 0.595 - ETA: 0s - loss: 1.6980 - main_output_loss: 0.7251 - aux_o

10368/13734 [=====================>........] - ETA: 5s - loss: 1.6874 - main_output_loss: 0.6673 - aux_output_loss: 1.0201 - main_output_acc: 0.7812 - aux_output_acc: 0.593 - ETA: 4s - loss: 1.5503 - main_output_loss: 0.6185 - aux_output_loss: 0.9318 - main_output_acc: 0.7812 - aux_output_acc: 0.633 - ETA: 4s - loss: 1.5684 - main_output_loss: 0.6349 - aux_output_loss: 0.9335 - main_output_acc: 0.7526 - aux_output_acc: 0.627 - ETA: 4s - loss: 1.5693 - main_output_loss: 0.6207 - aux_output_loss: 0.9486 - main_output_acc: 0.7647 - aux_output_acc: 0.621 - ETA: 4s - loss: 1.5766 - main_output_loss: 0.6312 - aux_output_loss: 0.9454 - main_output_acc: 0.7599 - aux_output_acc: 0.616 - ETA: 4s - loss: 1.5806 - main_output_loss: 0.6383 - aux_output_loss: 0.9423 - main_output_acc: 0.7581 - aux_output_acc: 0.620 - ETA: 3s - loss: 1.5950 - main_output_loss: 0.6402 - aux_output_loss: 0.9548 - main_output_acc: 0.7595 - aux_output_acc: 0.607 - ETA: 4s - loss: 1.5782 - main_output_loss: 0.6353 - aux_o

In [241]:
pie_text, pie_nlp, pie_head=transform_X_y('test 23')
model.predict([pie_text, pie_nlp, pie_head])

[array([[4.2391010e-03, 9.7573972e-01, 7.2533253e-04, 1.9295882e-02],
        [4.3448317e-03, 9.7498441e-01, 7.7947992e-04, 1.9891206e-02]],
       dtype=float32), array([[0.01043377, 0.9493693 , 0.00124083, 0.03895605],
        [0.01043377, 0.9493693 , 0.00124083, 0.03895605]], dtype=float32)]

In [134]:
def vectorize(X, y=None, length=1):
    main_input=[]
    auxiliary_input=[]
    auxiliary_input_2=[]
    for index, row in enumerate(X[:length]):
        doc=nlp(row)
        doc=input_duplicator(doc)
        plain_text=[]
        nlp_integers=[]
        heads=[]
        for token in doc:
            text=lmdb_vec_word(token.text)
            pos=pos_IDS.index(token.pos_)
            dep=' '+token.dep_
            dep=dep.strip().upper()
            if dep=='':
                dep=len(dep_labels)+1
            elif dep in dep_labels:
                dep=dep_labels.index(dep)
            else:
                dep=dep_labels.index(dep_mappings[dep])
            head=lmdb_vec_word(token.head.text)
            plain_text.append(text)
            nlp_integers.append([pos,dep])
            heads.append(head)
        main_input.append(plain_text)
        auxiliary_input.append(nlp_integers)
        auxiliary_input_2.append(heads)
    main_input=np.array(main_input)
    auxiliary_input=np.array(auxiliary_input)
    auxiliary_input_2=np.array(auxiliary_input_2)
    if y is not None:
        y_1=y[:length]
        return main_input,auxiliary_input, auxiliary_input_2, y_1
    else:
        return main_input,auxiliary_input,auxiliary_input_2

In [11]:
def input_duplicator(row):
    text_only=[token.text for token in row]
    doc=row
    #attrs=[POS, DEP, HEAD]
    while len(text_only)<7:
        orig_row=text_only.copy()
        for token in orig_row:
            text_only.append(token)
        doc=' '.join(text_only)
        doc=nlp(doc)
    return doc

In [136]:
def transform_X_y(X, y=None, length=1):
    main_input_1 = []
    auxiliary_input_1 =[]
    auxiliary_input_2_1=[]
    y_1= []
    if y is not None:
        main_input, auxiliary_input, auxiliary_input_2, y=vectorize(X, y, length)
    else:
        main_input, auxiliary_input, auxiliary_input_2=vectorize(X)
    for index in range(0, len(main_input)):
        doc=main_input[index]
        nlp=auxiliary_input[index]
        head=auxiliary_input_2[index]
        for i in range(6, len(doc)):
            main_input_1.append(doc[i-6:i])
            auxiliary_input_1.append(nlp[i-6:i])
            auxiliary_input_2_1.append(head[i-6:i])
            if y is not None:
                y_1.append(y.iloc[index])
    if y is not None:
        y_1=transform_y(y_1)
        return np.array(main_input_1), np.array(auxiliary_input_1), np.array(auxiliary_input_2_1), y_1
    else:
        return np.array(main_input_1), np.array(auxiliary_input_1), np.array(auxiliary_input_2_1)

In [307]:
def iterative_training(X, y, length, num_epochs):
    num_chunks=len(X)//length
    accs=[]
    for u in range(num_epochs):
        for i in range(num_chunks):
            print('Start of split ', i)
            X_1=X.iloc[i*length:(i+1)*length]
            y_1=y.iloc[i*length:(i+1)*length]
            main_train, aux_1_train, aux_2_train, y_1_train= transform_X_y(X_1, y_1, length)
            if i!=num_chunks-1:
                model.fit([main_train, aux_1_train, aux_2_train], [y_1_train,y_1_train])
            else:
                summary=model.evaluate([main_train, aux_1_train, aux_2_train], [y_1_train,y_1_train])
                accs.append(summary[1])
                print(summary)
                if u>1:
                    if accs[u]-accs[u-1]<-.02:
                        return 0
        #Appending the final X_test and y_test arrays only to save memory
        '''for i in range(len(X_test)):
            X_tests.append(X_test[i])
            y_tests.append(y_test[i])
    X_test=np.array(X_tests)
    y_test=np.array(y_tests)'''
    #return X_test, y_test

In [317]:
iterative_training(X_init, y_init, 500, 5)

Start of split  0
Epoch 1/1
9170/9170 [==============================] - ETA: 27:13 - loss: 2.7902 - main_output_loss: 1.4070 - aux_output_loss: 1.3832 - main_output_acc: 0.1250 - aux_output_acc: 0.34 - ETA: 4:29 - loss: 2.7600 - main_output_loss: 1.3772 - aux_output_loss: 1.3828 - main_output_acc: 0.3385 - aux_output_acc: 0.3073 - ETA: 2:13 - loss: 2.7655 - main_output_loss: 1.3803 - aux_output_loss: 1.3851 - main_output_acc: 0.3151 - aux_output_acc: 0.286 - ETA: 1:27 - loss: 2.7628 - main_output_loss: 1.3777 - aux_output_loss: 1.3852 - main_output_acc: 0.3160 - aux_output_acc: 0.289 - ETA: 1:04 - loss: 2.7686 - main_output_loss: 1.3833 - aux_output_loss: 1.3853 - main_output_acc: 0.2969 - aux_output_acc: 0.287 - ETA: 53s - loss: 2.7669 - main_output_loss: 1.3824 - aux_output_loss: 1.3845 - main_output_acc: 0.2996 - aux_output_acc: 0.285 - ETA: 43s - loss: 2.7606 - main_output_loss: 1.3791 - aux_output_loss: 1.3815 - main_output_acc: 0.3045 - aux_output_acc: 0.29 - ETA: 37s - loss: 2.

9416/9416 [==============================] - ETA: 4s - loss: 2.1913 - main_output_loss: 1.0925 - aux_output_loss: 1.0988 - main_output_acc: 0.6562 - aux_output_acc: 0.562 - ETA: 3s - loss: 2.3857 - main_output_loss: 1.2046 - aux_output_loss: 1.1812 - main_output_acc: 0.5052 - aux_output_acc: 0.453 - ETA: 3s - loss: 2.4268 - main_output_loss: 1.2137 - aux_output_loss: 1.2131 - main_output_acc: 0.5000 - aux_output_acc: 0.437 - ETA: 2s - loss: 2.4410 - main_output_loss: 1.2184 - aux_output_loss: 1.2226 - main_output_acc: 0.5000 - aux_output_acc: 0.439 - ETA: 2s - loss: 2.4448 - main_output_loss: 1.2198 - aux_output_loss: 1.2249 - main_output_acc: 0.4918 - aux_output_acc: 0.442 - ETA: 2s - loss: 2.4223 - main_output_loss: 1.1982 - aux_output_loss: 1.2241 - main_output_acc: 0.5022 - aux_output_acc: 0.445 - ETA: 2s - loss: 2.4011 - main_output_loss: 1.1815 - aux_output_loss: 1.2196 - main_output_acc: 0.5080 - aux_output_acc: 0.455 - ETA: 2s - loss: 2.3753 - main_output_loss: 1.1616 - aux_out

9108/9108 [==============================] - ETA: 3s - loss: 2.2602 - main_output_loss: 1.1271 - aux_output_loss: 1.1331 - main_output_acc: 0.5312 - aux_output_acc: 0.468 - ETA: 2s - loss: 2.3174 - main_output_loss: 1.1459 - aux_output_loss: 1.1715 - main_output_acc: 0.5208 - aux_output_acc: 0.489 - ETA: 2s - loss: 2.2435 - main_output_loss: 1.1120 - aux_output_loss: 1.1315 - main_output_acc: 0.5341 - aux_output_acc: 0.505 - ETA: 2s - loss: 2.2171 - main_output_loss: 1.0899 - aux_output_loss: 1.1272 - main_output_acc: 0.5404 - aux_output_acc: 0.507 - ETA: 2s - loss: 2.2272 - main_output_loss: 1.1038 - aux_output_loss: 1.1234 - main_output_acc: 0.5312 - aux_output_acc: 0.505 - ETA: 2s - loss: 2.2223 - main_output_loss: 1.1025 - aux_output_loss: 1.1198 - main_output_acc: 0.5300 - aux_output_acc: 0.508 - ETA: 2s - loss: 2.2258 - main_output_loss: 1.1040 - aux_output_loss: 1.1218 - main_output_acc: 0.5302 - aux_output_acc: 0.510 - ETA: 2s - loss: 2.2234 - main_output_loss: 1.0983 - aux_out

9423/9423 [==============================] - ETA: 3s - loss: 2.9836 - main_output_loss: 1.5267 - aux_output_loss: 1.4570 - main_output_acc: 0.3125 - aux_output_acc: 0.406 - ETA: 3s - loss: 2.3883 - main_output_loss: 1.2208 - aux_output_loss: 1.1675 - main_output_acc: 0.4688 - aux_output_acc: 0.487 - ETA: 3s - loss: 2.3024 - main_output_loss: 1.1432 - aux_output_loss: 1.1592 - main_output_acc: 0.5094 - aux_output_acc: 0.521 - ETA: 3s - loss: 2.2463 - main_output_loss: 1.1158 - aux_output_loss: 1.1305 - main_output_acc: 0.5332 - aux_output_acc: 0.531 - ETA: 2s - loss: 2.2813 - main_output_loss: 1.1339 - aux_output_loss: 1.1475 - main_output_acc: 0.5298 - aux_output_acc: 0.517 - ETA: 2s - loss: 2.2968 - main_output_loss: 1.1420 - aux_output_loss: 1.1548 - main_output_acc: 0.5216 - aux_output_acc: 0.502 - ETA: 2s - loss: 2.2467 - main_output_loss: 1.1152 - aux_output_loss: 1.1315 - main_output_acc: 0.5363 - aux_output_acc: 0.515 - ETA: 2s - loss: 2.2284 - main_output_loss: 1.1072 - aux_out

9662/9662 [==============================] - ETA: 3s - loss: 1.8797 - main_output_loss: 0.9261 - aux_output_loss: 0.9536 - main_output_acc: 0.6562 - aux_output_acc: 0.562 - ETA: 3s - loss: 2.0817 - main_output_loss: 1.0092 - aux_output_loss: 1.0725 - main_output_acc: 0.5875 - aux_output_acc: 0.506 - ETA: 3s - loss: 2.1940 - main_output_loss: 1.0723 - aux_output_loss: 1.1217 - main_output_acc: 0.5531 - aux_output_acc: 0.500 - ETA: 3s - loss: 2.2074 - main_output_loss: 1.0812 - aux_output_loss: 1.1262 - main_output_acc: 0.5488 - aux_output_acc: 0.496 - ETA: 2s - loss: 2.2046 - main_output_loss: 1.0834 - aux_output_loss: 1.1212 - main_output_acc: 0.5476 - aux_output_acc: 0.497 - ETA: 2s - loss: 2.1865 - main_output_loss: 1.0751 - aux_output_loss: 1.1114 - main_output_acc: 0.5451 - aux_output_acc: 0.501 - ETA: 2s - loss: 2.2135 - main_output_loss: 1.0899 - aux_output_loss: 1.1237 - main_output_acc: 0.5381 - aux_output_acc: 0.495 - ETA: 2s - loss: 2.2146 - main_output_loss: 1.0893 - aux_out

...

9423/9423 [==============================] - ETA: 4s - loss: 1.9527 - main_output_loss: 0.9742 - aux_output_loss: 0.9785 - main_output_acc: 0.5938 - aux_output_acc: 0.625 - ETA: 3s - loss: 2.1490 - main_output_loss: 1.0476 - aux_output_loss: 1.1014 - main_output_acc: 0.5875 - aux_output_acc: 0.537 - ETA: 3s - loss: 1.9604 - main_output_loss: 0.9339 - aux_output_loss: 1.0265 - main_output_acc: 0.6534 - aux_output_acc: 0.599 - ETA: 2s - loss: 1.9107 - main_output_loss: 0.9086 - aux_output_loss: 1.0021 - main_output_acc: 0.6636 - aux_output_acc: 0.602 - ETA: 2s - loss: 2.0119 - main_output_loss: 0.9766 - aux_output_loss: 1.0353 - main_output_acc: 0.6277 - aux_output_acc: 0.576 - ETA: 2s - loss: 1.9915 - main_output_loss: 0.9621 - aux_output_loss: 1.0294 - main_output_acc: 0.6347 - aux_output_acc: 0.585 - ETA: 2s - loss: 2.0071 - main_output_loss: 0.9717 - aux_output_loss: 1.0354 - main_output_acc: 0.6241 - aux_output_acc: 0.583 - ETA: 2s - loss: 2.0019 - main_output_loss: 0.9670 - aux_out

9662/9662 [==============================] - ETA: 3s - loss: 2.2953 - main_output_loss: 1.1994 - aux_output_loss: 1.0958 - main_output_acc: 0.5000 - aux_output_acc: 0.500 - ETA: 4s - loss: 2.2911 - main_output_loss: 1.1279 - aux_output_loss: 1.1632 - main_output_acc: 0.5625 - aux_output_acc: 0.506 - ETA: 3s - loss: 2.1327 - main_output_loss: 1.0402 - aux_output_loss: 1.0925 - main_output_acc: 0.5687 - aux_output_acc: 0.528 - ETA: 3s - loss: 2.0903 - main_output_loss: 1.0160 - aux_output_loss: 1.0743 - main_output_acc: 0.5750 - aux_output_acc: 0.541 - ETA: 3s - loss: 2.0853 - main_output_loss: 1.0132 - aux_output_loss: 1.0720 - main_output_acc: 0.5833 - aux_output_acc: 0.547 - ETA: 2s - loss: 2.0919 - main_output_loss: 1.0126 - aux_output_loss: 1.0793 - main_output_acc: 0.5845 - aux_output_acc: 0.538 - ETA: 2s - loss: 2.1167 - main_output_loss: 1.0248 - aux_output_loss: 1.0919 - main_output_acc: 0.5758 - aux_output_acc: 0.529 - ETA: 2s - loss: 2.0936 - main_output_loss: 1.0090 - aux_out

9170/9170 [==============================] - ETA: 3s - loss: 1.8217 - main_output_loss: 0.9707 - aux_output_loss: 0.8509 - main_output_acc: 0.5312 - aux_output_acc: 0.656 - ETA: 2s - loss: 2.1392 - main_output_loss: 1.0502 - aux_output_loss: 1.0890 - main_output_acc: 0.5268 - aux_output_acc: 0.540 - ETA: 2s - loss: 2.1696 - main_output_loss: 1.0615 - aux_output_loss: 1.1081 - main_output_acc: 0.5721 - aux_output_acc: 0.550 - ETA: 2s - loss: 2.1695 - main_output_loss: 1.0610 - aux_output_loss: 1.1085 - main_output_acc: 0.5576 - aux_output_acc: 0.541 - ETA: 2s - loss: 2.1622 - main_output_loss: 1.0539 - aux_output_loss: 1.1084 - main_output_acc: 0.5675 - aux_output_acc: 0.537 - ETA: 2s - loss: 2.1529 - main_output_loss: 1.0493 - aux_output_loss: 1.1036 - main_output_acc: 0.5665 - aux_output_acc: 0.539 - ETA: 2s - loss: 2.1273 - main_output_loss: 1.0328 - aux_output_loss: 1.0946 - main_output_acc: 0.5752 - aux_output_acc: 0.542 - ETA: 2s - loss: 2.0987 - main_output_loss: 1.0142 - aux_out

9416/9416 [==============================] - ETA: 3s - loss: 1.9948 - main_output_loss: 1.0350 - aux_output_loss: 0.9598 - main_output_acc: 0.6562 - aux_output_acc: 0.593 - ETA: 3s - loss: 2.1003 - main_output_loss: 1.0744 - aux_output_loss: 1.0260 - main_output_acc: 0.6250 - aux_output_acc: 0.557 - ETA: 2s - loss: 2.0635 - main_output_loss: 1.0263 - aux_output_loss: 1.0372 - main_output_acc: 0.6172 - aux_output_acc: 0.549 - ETA: 2s - loss: 2.0437 - main_output_loss: 1.0055 - aux_output_loss: 1.0382 - main_output_acc: 0.6042 - aux_output_acc: 0.543 - ETA: 2s - loss: 2.0655 - main_output_loss: 1.0175 - aux_output_loss: 1.0480 - main_output_acc: 0.6003 - aux_output_acc: 0.540 - ETA: 2s - loss: 2.0019 - main_output_loss: 0.9778 - aux_output_loss: 1.0241 - main_output_acc: 0.6115 - aux_output_acc: 0.558 - ETA: 2s - loss: 1.9917 - main_output_loss: 0.9701 - aux_output_loss: 1.0215 - main_output_acc: 0.6102 - aux_output_acc: 0.554 - ETA: 2s - loss: 2.0076 - main_output_loss: 0.9751 - aux_out

KeyboardInterrupt: 

In [305]:
model.metrics_names

['loss',
 'main_output_loss',
 'aux_output_loss',
 'main_output_acc',
 'aux_output_acc']

In [81]:
four_emotions_2.save('models/four_emotions_2_softmax.h5')

In [478]:
X.shape

(1823, 6, 4, 300)

In [461]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [293]:
four_emotions_2.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'categorical_crossentropy')

In [292]:
four_emotions_2=create_lstm()

In [473]:
four_emotions_2.predict(transform_X_y('I like pie'))

array([[-0.05093911,  0.04795563,  0.03047078, -0.03990684],
       [-0.0503883 ,  0.04763907,  0.03028065, -0.03962052]],
      dtype=float32)

In [33]:
pos_IDS = [
    "NO_TAG",
    "ADJ",
    "ADP",
    "ADV",
    "AUX",
    "CONJ",
    "CCONJ",
    "DET",
    "INTJ",
    "NOUN",
    "NUM",
    "PART",
    "PRON",
    "PROPN",
    "PUNCT",
    "SCONJ",
    "SYM",
    "VERB",
    "X",
    "EOL",
    "SPACE"]

In [22]:
dep_table='''Label	Description	Since
ACL	Clausal modifier of noun	3.1.0
ACOMP	Adjectival complement	1.0.0
ADVCL	Adverbial clause modifier	1.0.0
ADVMOD	Adverbial modifier	1.0.0
AGENT	Agent	1.0.0
AMOD	Adjectival modifier	1.0.0
APPOS	Appositional modifier	1.0.0
ATTR	Attribute	1.0.0
AUX	Auxiliary	1.0.0
AUXPASS	Auxiliary (passive)	1.0.0
CASE	Case marker	3.1.0
CC	Coordinating conjunction	1.0.0
CCOMP	Clausal complement	1.0.0
COMPOUND	Compound modifier	3.1.0
CONJ	Conjunct	1.0.0
CSUBJ	Clausal subject	1.0.0
CSUBJPASS	Clausal subject (passive)	1.0.0
DATIVE	Dative	3.1.0
DEP	Unclassified dependent	1.0.0
DET	Determiner	1.0.0
DOBJ	Direct Object	1.0.0
EXPL	Expletive	1.0.0
INTJ	Interjection	1.0.0
MARK	Marker	1.0.0
META	Meta modifier	1.0.0
NEG	Negation modifier	1.0.0
NOUNMOD	Modifier of nominal	3.1.0
NPMOD	Noun phrase as adverbial modifier	3.1.0
NSUBJ	Nominal subject	1.0.0
NSUBJPASS	Nominal subject (passive)	1.0.0
NUMMOD	Number modifier	3.1.0
OPRD	Object predicate	1.0.0
PARATAXIS	Parataxis	1.0.0
PCOMP	Complement of preposition	1.0.0
POBJ	Object of preposition	1.0.0
POSS	Possession modifier	1.0.0
PRECONJ	Pre-correlative conjunction	1.0.0
PREDET	Pre-determiner	1.0.0
PREP	Prepositional modifier	1.0.0
PRT	Particle	1.0.0
PUNCT	Punctuation	1.0.0
QUANTMOD	Modifier of quantifier	1.0.0
RELCL	Relative clause modifier	3.1.0
ROOT	Root	1.0.0
XCOMP	Open clausal complement	1.0.0'''

In [23]:
dep_deprecated='''COMPLM	Complementizer	3.0.0	Replaced to mark
INFMOD	Infinitival modifier	3.0.0	Replaced to acl
PARTMOD	Participal modifier	3.0.0	Replaced to acl
HMOD	Modifier in hyphenation	3.1.0	Replaced to compound
HYPH	Hyphen	3.1.0	Replaced to punct
IOBJ	Indirect object	3.1.0	Replaced to dative
NUM	Number modifier	3.1.0	Replaced to nummod
NUMBER	Number compound modifier	3.1.0	Replaced to compound
NMOD	Modifier of nominal	3.1.0	Replaced to nounmod
NN	Noun compound modifier	3.1.0	Replaced to compound
NPADVMOD	Noun phrase as adverbial modifier	3.1.0	Replaced to npmod
POSSESSIVE	Possessive modifier	3.1.0	Replaced to case
RCMOD	Relative clause modifier	3.1.0	Replaced to relcl'''

In [24]:
def parse_table(table):
    dep_allterms=table.split('\t')
    dep_labels=[]
    for term in dep_allterms:
        splits=term.split('\n')
        for subterm in splits:
            if subterm==subterm.upper():
                #check for numbers
                if '0' not in subterm:
                    dep_labels.append(subterm)
    return dep_labels

In [71]:
dep_labels=parse_table(dep_table)
dep_labels

['ACL',
 'ACOMP',
 'ADVCL',
 'ADVMOD',
 'AGENT',
 'AMOD',
 'APPOS',
 'ATTR',
 'AUX',
 'AUXPASS',
 'CASE',
 'CC',
 'CCOMP',
 'COMPOUND',
 'CONJ',
 'CSUBJ',
 'CSUBJPASS',
 'DATIVE',
 'DEP',
 'DET',
 'DOBJ',
 'EXPL',
 'INTJ',
 'MARK',
 'META',
 'NEG',
 'NOUNMOD',
 'NPMOD',
 'NSUBJ',
 'NSUBJPASS',
 'NUMMOD',
 'OPRD',
 'PARATAXIS',
 'PCOMP',
 'POBJ',
 'POSS',
 'PRECONJ',
 'PREDET',
 'PREP',
 'PRT',
 'PUNCT',
 'QUANTMOD',
 'RELCL',
 'ROOT',
 'XCOMP']

In [26]:
def parse_replace(table):
    dep_allterms=table.split('\t')
    dep_labels=[]
    replacements=[]
    for term in dep_allterms:
        splits=term.split('\n')
        for subterm in splits:
            if subterm==subterm.upper():
                #check for numbers
                if '0' not in subterm:
                    dep_labels.append(subterm)
            if 'Replaced' in subterm:
                replacement=subterm.split('Replaced to')[1].strip()
                replacements.append(replacement.upper())
    replace_dict={}
    for index, label in enumerate(dep_labels):
        replace_dict[label]=replacements[index]
    return replace_dict

In [35]:
dep_mappings=parse_replace(dep_deprecated)
dep_mappings

{'COMPLM': 'MARK',
 'HMOD': 'COMPOUND',
 'HYPH': 'PUNCT',
 'INFMOD': 'ACL',
 'IOBJ': 'DATIVE',
 'NMOD': 'NOUNMOD',
 'NN': 'COMPOUND',
 'NPADVMOD': 'NPMOD',
 'NUM': 'NUMMOD',
 'NUMBER': 'COMPOUND',
 'PARTMOD': 'ACL',
 'POSSESSIVE': 'CASE',
 'RCMOD': 'RELCL'}

In [36]:
dep_deprecated_labels=parse_table(dep_deprecated)
dep_deprecated_labels

['COMPLM',
 'INFMOD',
 'PARTMOD',
 'HMOD',
 'HYPH',
 'IOBJ',
 'NUM',
 'NUMBER',
 'NMOD',
 'NN',
 'NPADVMOD',
 'POSSESSIVE',
 'RCMOD']

In [37]:
dep_labels

['ACL',
 'ACOMP',
 'ADVCL',
 'ADVMOD',
 'AGENT',
 'AMOD',
 'APPOS',
 'ATTR',
 'AUX',
 'AUXPASS',
 'CASE',
 'CC',
 'CCOMP',
 'COMPOUND',
 'CONJ',
 'CSUBJ',
 'CSUBJPASS',
 'DATIVE',
 'DEP',
 'DET',
 'DOBJ',
 'EXPL',
 'INTJ',
 'MARK',
 'META',
 'NEG',
 'NOUNMOD',
 'NPMOD',
 'NSUBJ',
 'NSUBJPASS',
 'NUMMOD',
 'OPRD',
 'PARATAXIS',
 'PCOMP',
 'POBJ',
 'POSS',
 'PRECONJ',
 'PREDET',
 'PREP',
 'PRT',
 'PUNCT',
 'QUANTMOD',
 'RELCL',
 'ROOT',
 'XCOMP']

In [30]:
dep_mappings=transform_y(dep_labels)

In [110]:
dep_mappings.shape

(45, 45)

In [243]:
def create_lstm():

     # Initialising the RNN
    regressor = Sequential()

    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units=25, return_sequences= True, input_shape=(6, 300)))
    #regressor.add(TimeDistributed(LSTM(units = 30, return_sequences = False), input_shape=(6,4,300)))
    regressor.add(Dropout(0.4))

    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 30, return_sequences = True, recurrent_dropout=0.3))
    regressor.add(Dropout(0.4))

    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 30, return_sequences = True))
    regressor.add(Dropout(0.4))

    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 30, return_sequences = False))
    regressor.add(Dropout(0.4))


    # Reshape
    #regressor.add(Reshape((-1, 6,4)))

    # Flatten
    #regressor.add(Flatten())

    # Adding the output layer
    regressor.add(Dense(units = 4, activation='softmax'))

    return regressor

In [253]:
def vectorize_1(X, y=None,length=1):
    X_1=[]
    for index, row in enumerate(X[:length]):
        doc=nlp(row)
        doc=input_duplicator(doc)
        row_arr=[]
        for token in doc:
            text=lmdb_vec_word(token.text)
            pos=pos_IDS.index(token.pos_)
            zeros=np.zeros(300, dtype='float32')
            zeros[pos]=1
            pos=zeros
            dep=' '+token.dep_
            dep=dep.strip().upper()
            if dep=='':
                dep=len(dep_labels)+1
            elif dep in dep_labels:
                dep=dep_labels.index(dep)
            else:
                dep=dep_labels.index(dep_mappings[dep])
            zeros=np.zeros(300, dtype='float32')
            zeros[dep]=1
            dep=zeros
            head=lmdb_vec_word(token.head.text)
            #token_li=[text,pos,dep,head]
            row_arr=np.reshape(token_li, newshape=(1200))
        X_1.append(row_arr)
    X_1=np.array(X_1)
    if y is not None:
        y_1=y[:length]
        return X_1, y_1
    else:
        return X_1

In [259]:
def vectorize_text(X, y=None,length=1):
    X_1=[]
    for index, row in enumerate(X[:length]):
        doc=nlp(row)
        doc=input_duplicator(doc)
        row_arr=[]
        for token in doc:
            text=lmdb_vec_word(token.text)
            row_arr.append(text)
        X_1.append(row_arr)
    X_1=np.array(X_1)
    if y is not None:
        y_1=y[:length]
        return X_1, y_1
    else:
        return X_1

In [269]:
def transform_X_y_1(X, y=None, length=1):
    X_1 = []
    y_1= []
    if y is not None:
        X, y=vectorize_text(X, y, length)
    else:
        X=vectorize_text(X, length=length)
    for index in range(0, len(X)):
        doc=X[index]
        for i in range(6, len(doc)):
            X_1.append(doc[i-6:i])
            if y is not None:
                y_1.append(y.iloc[index])
    if y is not None:
        y_1=transform_y(y_1)
        return np.array(X_1), y_1
    else:
        return np.array(X_1)

In [295]:
def iterative_training_1(X, y, length, num_epochs):
    num_chunks=len(X)//length
    accs=[]
    for u in range(num_epochs):
        for i in range(num_chunks):
            print('Start of split ', i)
            X_1=X.iloc[i*length:(i+1)*length]
            y_1=y.iloc[i*length:(i+1)*length]
            X_1, y_1= transform_X_y_1(X_1, y_1, length)
            if i!=num_chunks-1:
                four_emotions_2.fit(X_1, y_1, epochs=1)
            else:
                summary=four_emotions_2.evaluate(X_1, y_1)
                accs.append(summary[1])
                print(summary)
                if u>1:
                    if accs[u]-accs[u-1]<-.02:
                        return 0
        #Appending the final X_test and y_test arrays only to save memory
        '''for i in range(len(X_test)):
            X_tests.append(X_test[i])
            y_tests.append(y_test[i])
    X_test=np.array(X_tests)
    y_test=np.array(y_tests)'''
    #return X_test, y_test

In [270]:
X_1=transform_X_y_1(X_init, length=1000)

In [271]:
X_1.shape

(18313, 6, 300)

In [273]:
X_1_train, X_1_test=train_test_split(X_1, random_state=1)

In [276]:
regressor=create_lstm()

In [277]:
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'categorical_crossentropy')

In [280]:
regressor.fit(X_1_train, y_1_train, epochs=5)

Epoch 1/5
13734/13734 [==============================] - ETA: 9s - loss: 1.2338 - acc: 0.500 - ETA: 8s - loss: 1.2369 - acc: 0.468 - ETA: 8s - loss: 1.2515 - acc: 0.459 - ETA: 7s - loss: 1.2293 - acc: 0.465 - ETA: 7s - loss: 1.2163 - acc: 0.473 - ETA: 7s - loss: 1.2169 - acc: 0.472 - ETA: 7s - loss: 1.2107 - acc: 0.478 - ETA: 7s - loss: 1.2147 - acc: 0.474 - ETA: 7s - loss: 1.2189 - acc: 0.463 - ETA: 7s - loss: 1.2166 - acc: 0.467 - ETA: 7s - loss: 1.2191 - acc: 0.463 - ETA: 7s - loss: 1.2175 - acc: 0.462 - ETA: 7s - loss: 1.2184 - acc: 0.458 - ETA: 7s - loss: 1.2190 - acc: 0.455 - ETA: 6s - loss: 1.2167 - acc: 0.452 - ETA: 6s - loss: 1.2139 - acc: 0.456 - ETA: 6s - loss: 1.2146 - acc: 0.457 - ETA: 6s - loss: 1.2125 - acc: 0.459 - ETA: 6s - loss: 1.2129 - acc: 0.460 - ETA: 6s - loss: 1.2051 - acc: 0.466 - ETA: 6s - loss: 1.2098 - acc: 0.461 - ETA: 6s - loss: 1.2090 - acc: 0.461 - ETA: 6s - loss: 1.2075 - acc: 0.463 - ETA: 6s - loss: 1.2019 - acc: 0.466 - ETA: 5s - loss: 1.2031 - acc: 0

13734/13734 [==============================] - ETA: 7s - loss: 0.7373 - acc: 0.781 - ETA: 7s - loss: 0.9377 - acc: 0.625 - ETA: 7s - loss: 1.0066 - acc: 0.620 - ETA: 7s - loss: 1.0266 - acc: 0.606 - ETA: 7s - loss: 1.0193 - acc: 0.620 - ETA: 7s - loss: 1.0415 - acc: 0.591 - ETA: 7s - loss: 1.0397 - acc: 0.593 - ETA: 7s - loss: 1.0432 - acc: 0.596 - ETA: 7s - loss: 1.0465 - acc: 0.593 - ETA: 6s - loss: 1.0578 - acc: 0.576 - ETA: 6s - loss: 1.0578 - acc: 0.571 - ETA: 6s - loss: 1.0631 - acc: 0.565 - ETA: 6s - loss: 1.0700 - acc: 0.564 - ETA: 6s - loss: 1.0732 - acc: 0.560 - ETA: 6s - loss: 1.0796 - acc: 0.559 - ETA: 6s - loss: 1.0769 - acc: 0.559 - ETA: 6s - loss: 1.0762 - acc: 0.558 - ETA: 6s - loss: 1.0773 - acc: 0.555 - ETA: 6s - loss: 1.0732 - acc: 0.558 - ETA: 6s - loss: 1.0731 - acc: 0.558 - ETA: 6s - loss: 1.0768 - acc: 0.557 - ETA: 5s - loss: 1.0770 - acc: 0.556 - ETA: 5s - loss: 1.0775 - acc: 0.555 - ETA: 5s - loss: 1.0775 - acc: 0.553 - ETA: 5s - loss: 1.0783 - acc: 0.552 - ETA

13734/13734 [==============================] - ETA: 7s - loss: 1.0193 - acc: 0.531 - ETA: 7s - loss: 1.0531 - acc: 0.546 - ETA: 7s - loss: 0.9939 - acc: 0.584 - ETA: 7s - loss: 1.0338 - acc: 0.575 - ETA: 7s - loss: 1.0332 - acc: 0.567 - ETA: 7s - loss: 1.0138 - acc: 0.584 - ETA: 7s - loss: 1.0012 - acc: 0.596 - ETA: 7s - loss: 1.0249 - acc: 0.578 - ETA: 6s - loss: 1.0298 - acc: 0.577 - ETA: 6s - loss: 1.0321 - acc: 0.580 - ETA: 7s - loss: 1.0260 - acc: 0.583 - ETA: 6s - loss: 1.0127 - acc: 0.589 - ETA: 6s - loss: 1.0130 - acc: 0.589 - ETA: 6s - loss: 1.0045 - acc: 0.596 - ETA: 6s - loss: 1.0027 - acc: 0.601 - ETA: 6s - loss: 1.0051 - acc: 0.601 - ETA: 6s - loss: 1.0078 - acc: 0.601 - ETA: 6s - loss: 1.0028 - acc: 0.599 - ETA: 6s - loss: 1.0068 - acc: 0.599 - ETA: 6s - loss: 1.0028 - acc: 0.603 - ETA: 6s - loss: 1.0082 - acc: 0.602 - ETA: 6s - loss: 1.0120 - acc: 0.599 - ETA: 6s - loss: 1.0076 - acc: 0.601 - ETA: 6s - loss: 1.0088 - acc: 0.600 - ETA: 6s - loss: 1.0102 - acc: 0.598 - ETA

In [290]:
regressor.predict(transform_X_y_1('sadness'))

array([[0.0282448 , 0.9014813 , 0.00984489, 0.060429  ],
       [0.0282448 , 0.9014813 , 0.00984489, 0.060429  ]], dtype=float32)

In [31]:
pos_mappings=transform_y(pos_IDS)

<h3>Results of training with flattend array of input size (6,1200)</h3>
Test accuracy did not exceed 56% while training accuracy continued to improve. This is inferior to four_emotions model 1. 

<h3>Anger, fear, joy, sadness</h3>